Simple Models for Target Columns

In [3]:
import sensor_imputation_thesis.shared.load_data as load 
from data_insight import setup_duckdb
from duckdb import DuckDBPyConnection as DuckDB
import pandas as pd
from duckdb import DuckDBPyRelation as Relation
from pathlib import Path
import hashlib
from sklearn.pipeline import Pipeline 
import numpy as np
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score,root_mean_squared_error
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.compose import make_column_selector as selector
import mlflow
import mlflow.sklearn
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns
import matplotlib.pyplot as plt

pid for the models of first two features below is "cdb73a020c59f7745d85e04d85bb8521"

In [2]:
#this df is the one with 9 missing cols and important cols that contribute to the variance of 9 cols 
df=pd.read_parquet("/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/df_for_simplemodels.parquet")

In [3]:
print(df.columns)

Index(['time', 'fr_eng', 'te_exh_cyl_out__0', 'pd_air_ic__0',
       'pr_exh_turb_out__0', 'te_air_ic_out__0', 'te_seawater',
       'te_air_comp_in_a__0', 'te_air_comp_in_b__0', 'fr_tc__0', 'pr_baro',
       'pd_air_ic__0_1', 'pr_exh_rec', 'te_exh_turb_in__0',
       'te_exh_turb_out__0', 'bo_aux_blower_running', 're_eng_load',
       'pr_air_scav_ecs', 'pr_air_scav', 'te_air_scav_rec',
       'te_air_ic_out__0_1', 'pr_cyl_comp__0', 'pr_cyl_max__0', 'se_mip__0',
       'te_exh_cyl_out__0_1', 'fr_eng_setpoint', 'te_air_scav_rec_iso',
       'pr_cyl_max_mv_iso', 'pr_cyl_comp_mv_iso', 'fr_eng_ecs',
       'pr_air_scav_iso', 'engine_type_G80ME-C9.5-GI-LPSCR'],
      dtype='object')


In [4]:
len(df)

383312

In [6]:
#drop Nans because the later processes don't accept nan values
df=df.dropna(axis=1,how="all")
df=df.dropna(axis=0,how="any")
numeric_df=df.select_dtypes(include='number')
print(len(numeric_df),numeric_df.columns)

188024 Index(['fr_eng', 'te_exh_cyl_out__0', 'pd_air_ic__0', 'te_air_ic_out__0',
       'te_seawater', 'pr_baro', 'pd_air_ic__0_1', 'pr_exh_rec',
       'bo_aux_blower_running', 're_eng_load', 'pr_air_scav_ecs',
       'pr_air_scav', 'te_air_scav_rec', 'te_air_ic_out__0_1',
       'pr_cyl_comp__0', 'pr_cyl_max__0', 'se_mip__0', 'te_exh_cyl_out__0_1',
       'fr_eng_setpoint', 'te_air_scav_rec_iso', 'pr_cyl_max_mv_iso',
       'pr_cyl_comp_mv_iso', 'fr_eng_ecs', 'pr_air_scav_iso',
       'engine_type_G80ME-C9.5-GI-LPSCR'],
      dtype='object')


In [7]:
len(numeric_df.columns)

25

In [24]:
numeric_df.describe()

,fr_eng,te_exh_cyl_out__0,pd_air_ic__0,te_air_ic_out__0,te_seawater,pr_baro,pd_air_ic__0_1,pr_exh_rec,bo_aux_blower_running,re_eng_load,...,pr_cyl_max__0,se_mip__0,te_exh_cyl_out__0_1,fr_eng_setpoint,te_air_scav_rec_iso,pr_cyl_max_mv_iso,pr_cyl_comp_mv_iso,fr_eng_ecs,pr_air_scav_iso,engine_type_G80ME-C9.5-GI-LPSCR
count,188024.000000,188024.000000,188024.000000,188024.000000,1.880240e+05,188024.000000,188024.000000,188024.000000,188024.000000,188024.000000,...,1.880240e+05,1.880240e+05,188024.000000,188024.000000,188024.000000,1.880240e+05,1.880240e+05,188024.000000,188024.000000,188024.0
mean,0.838775,533.302757,3151.454814,307.083712,2.731500e+02,101459.131281,3151.454814,89021.874867,0.254904,0.483343,...,1.446284e+07,9.517589e+05,533.302757,0.838329,304.475587,1.456114e+07,1.077625e+07,0.838775,99833.635388,1.0
std,0.130500,34.974583,1905.629263,1.868717,5.684357e-14,594.499761,1905.629263,48355.265269,0.435808,0.176496,...,2.392325e+06,2.555563e+05,34.974583,0.130999,2.475842,2.466624e+06,2.250869e+06,0.130500,52355.439774,0.0
min,0.167625,325.150000,0.000000,294.350000,2.731500e+02,94000.000000,0.000000,4000.000000,0.000000,0.000000,...,4.057249e+06,-3.486891e+04,325.150000,0.000000,299.056997,4.026332e+06,4.055897e+06,0.167625,597.633930,1.0
25%,0.786876,515.150000,1750.000000,305.850000,2.731500e+02,101100.000000,1750.000000,53000.000000,0.000000,0.370000,...,1.287457e+07,8.104598e+05,515.150000,0.786643,302.687664,1.300431e+07,9.346316e+06,0.786876,60194.674426,1.0
50%,0.884118,543.150000,3350.000000,306.950000,2.731500e+02,101400.000000,3350.000000,93000.000000,0.000000,0.530000,...,1.531530e+07,1.022972e+06,543.150000,0.883302,303.826959,1.551455e+07,1.157711e+07,0.884118,106705.193931,1.0
75%,0.917148,554.150000,4350.000000,308.150000,2.731500e+02,101800.000000,4350.000000,117000.000000,1.000000,0.590000,...,1.609056e+07,1.107588e+06,554.150000,0.916635,307.118440,1.628848e+07,1.227609e+07,0.917148,131549.740285,1.0
max,1.022311,597.150000,10000.000000,315.050000,2.731500e+02,103100.000000,10000.000000,211000.000000,1.000000,0.870000,...,1.891224e+07,1.524548e+06,597.150000,1.016633,310.687664,1.909924e+07,1.560651e+07,1.022311,229136.394211,1.0


In [25]:
numeric_df["te_air_scav_rec"].describe()

count    188024.000000
mean        311.743186
std           2.296181
min         304.950000
25%         309.950000
50%         311.450000
75%         313.450000
max         327.950000
Name: te_air_scav_rec, dtype: float64

Mlflow Setup 

In [17]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("SimpleModelsforTargetCols")

<Experiment: artifact_location='/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/mlruns/10', creation_time=1748089768112, experiment_id='10', last_update_time=1748089768112, lifecycle_stage='active', name='SimpleModelsforTargetCols', tags={}>

1. For pr_baro

In [8]:
y=numeric_df["pr_baro"]
X =numeric_df.drop(columns=["pr_baro"])

In [9]:
total_samples=len(X)
test_ratio=0.2
test_size=int(total_samples*test_ratio)
n_splits=(total_samples-test_size)//test_size
tscv = TimeSeriesSplit(n_splits=n_splits,test_size=test_size,gap=0)

for train_idx, test_idx in tscv.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]



Apply Standard Scaler & Linear Pipeline

In [10]:
Pipeline1 = Pipeline([ ('scaler', StandardScaler()), 
                    ('Linear Regression', LinearRegression())])

Pipeline1.fit(X_train,y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('Linear Regression', LinearRegression())])

In [11]:
input_example=X_test.iloc[:1]
with mlflow.start_run(run_name="linear_for_prbaro"):
    y_pred=Pipeline1.predict(X_test)
    mse=mean_squared_error(y_test,y_pred)
    rmse=root_mean_squared_error(y_test,y_pred)
    mae=mean_absolute_error(y_test,y_pred)
    r2=r2_score(y_test,y_pred)
    print("mse:",mse)
    print("rmse:",rmse)
    print("mae:",mae)
    print("r2:",r2)

    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2",r2)
    mlflow.log_metric("mse",mse)
    mlflow.log_metric("rmse",rmse)
    mlflow.sklearn.log_model(Pipeline1, artifact_path="linear_for_prbaro", input_example=input_example)

mse: 669297.7128362441
rmse: 818.1061745496388
mae: 699.6633202408862
r2: -0.7489316521460203
🏃 View run linear_for_prbaro at: http://localhost:5000/#/experiments/10/runs/c7bc64fa798a4215a8ae47e46450e255
🧪 View experiment at: http://localhost:5000/#/experiments/10


Linear with important features 

In [8]:
numeric_df.columns

Index(['fr_eng', 'te_exh_cyl_out__0', 'pd_air_ic__0', 'te_air_ic_out__0',
       'te_seawater', 'pr_baro', 'pd_air_ic__0_1', 'pr_exh_rec',
       'bo_aux_blower_running', 're_eng_load', 'pr_air_scav_ecs',
       'pr_air_scav', 'te_air_scav_rec', 'te_air_ic_out__0_1',
       'pr_cyl_comp__0', 'pr_cyl_max__0', 'se_mip__0', 'te_exh_cyl_out__0_1',
       'fr_eng_setpoint', 'te_air_scav_rec_iso', 'pr_cyl_max_mv_iso',
       'pr_cyl_comp_mv_iso', 'fr_eng_ecs', 'pr_air_scav_iso',
       'engine_type_G80ME-C9.5-GI-LPSCR'],
      dtype='object')

In [11]:
print(numeric_df.columns.tolist())


['fr_eng', 'te_exh_cyl_out__0', 'pd_air_ic__0', 'te_air_ic_out__0', 'te_seawater', 'pr_baro', 'pd_air_ic__0_1', 'pr_exh_rec', 'bo_aux_blower_running', 're_eng_load', 'pr_air_scav_ecs', 'pr_air_scav', 'te_air_scav_rec', 'te_air_ic_out__0_1', 'pr_cyl_comp__0', 'pr_cyl_max__0', 'se_mip__0', 'te_exh_cyl_out__0_1', 'fr_eng_setpoint', 'te_air_scav_rec_iso', 'pr_cyl_max_mv_iso', 'pr_cyl_comp_mv_iso', 'fr_eng_ecs', 'pr_air_scav_iso', 'engine_type_G80ME-C9.5-GI-LPSCR']


In [12]:
y=numeric_df["pr_baro"]
X =numeric_df[["te_seawater", "pr_cyl_max_mv_iso","fr_eng_setpoint","pr_cyl_max__0",
              "te_exh_cyl_out__0","pr_cyl_comp__0","pr_air_scav_ecs","pd_air_ic__0",
              "te_air_scav_rec","pr_cyl_comp_mv_iso","se_mip__0",
              "pd_air_ic__0_1","te_air_ic_out__0","pr_air_scav","fr_eng_ecs","pr_air_scav_iso",
              "fr_eng"]]
            

In [14]:
total_samples=len(X)
test_ratio=0.2
test_size=int(total_samples*test_ratio)
n_splits=(total_samples-test_size)//test_size
tscv = TimeSeriesSplit(n_splits=n_splits,test_size=test_size,gap=0)

for train_idx, test_idx in tscv.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]


In [15]:
Pipeline1 = Pipeline([ ('scaler', StandardScaler()), 
                    ('Linear Regression', LinearRegression())])

Pipeline1.fit(X_train,y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('Linear Regression', LinearRegression())])

In [18]:
input_example=X_test.iloc[:1]
with mlflow.start_run(run_name="linearwizimportfea_for_prbaro"):
    y_pred=Pipeline1.predict(X_test)
    mse=mean_squared_error(y_test,y_pred)
    rmse=root_mean_squared_error(y_test,y_pred)
    mae=mean_absolute_error(y_test,y_pred)
    r2=r2_score(y_test,y_pred)
    print("mse:",mse)
    print("rmse:",rmse)
    print("mae:",mae)
    print("r2:",r2)

    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2",r2)
    mlflow.log_metric("mse",mse)
    mlflow.log_metric("rmse",rmse)
    mlflow.sklearn.log_model(Pipeline1, artifact_path="linearwizimportfea_for_prbaro", input_example=input_example)

mse: 631417.6407823937
rmse: 794.617921256747
mae: 680.5247675072193
r2: -0.6499478132205765
🏃 View run linearwizimportfea_for_prbaro at: http://localhost:5000/#/experiments/10/runs/95a97f3c20704e6ba3bf4c7f3e2ed3e0
🧪 View experiment at: http://localhost:5000/#/experiments/10


Random Forest 

In [14]:
Pipeline2 = Pipeline([('Scaler', StandardScaler()), 
                    ('RandomForest', RandomForestRegressor())])

Pipeline2.fit(X_train,y_train)

with mlflow.start_run(run_name="rf_forprbaro"):
    y_pred=Pipeline2.predict(X_test)
    mse=mean_squared_error(y_test,y_pred)
    mae=mean_absolute_error(y_test,y_pred)
    rmse=root_mean_squared_error(y_test,y_pred)
    r2=r2_score(y_test,y_pred)
    print("mse:",mse)
    print("mae:",mae)
    print("rmse",rmse)
    print("r2:",r2)

    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2",r2)
    mlflow.log_metric("mse",mse)
    mlflow.log_metric("rmse",rmse)
    mlflow.sklearn.log_model(Pipeline2, artifact_path="rf_for_prbaro", input_example=input_example)


mse: 1036944.7546537602
mae: 800.2192851824274
rmse 1018.3048436758809
r2: -1.7096245335361955
🏃 View run rf_forprbaro at: http://localhost:5000/#/experiments/10/runs/359e7031233140219e95c0389ad3b342
🧪 View experiment at: http://localhost:5000/#/experiments/10


RF with important features as X 

In [19]:
Pipeline2 = Pipeline([('Scaler', StandardScaler()), 
                    ('RandomForest', RandomForestRegressor())])

Pipeline2.fit(X_train,y_train)

with mlflow.start_run(run_name="rfwizimportfea_forprbaro"):
    y_pred=Pipeline2.predict(X_test)
    mse=mean_squared_error(y_test,y_pred)
    mae=mean_absolute_error(y_test,y_pred)
    rmse=root_mean_squared_error(y_test,y_pred)
    r2=r2_score(y_test,y_pred)
    print("mse:",mse)
    print("mae:",mae)
    print("rmse",rmse)
    print("r2:",r2)

    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2",r2)
    mlflow.log_metric("mse",mse)
    mlflow.log_metric("rmse",rmse)
    mlflow.sklearn.log_model(Pipeline2, artifact_path="rfwizimortfea_for_prbaro", input_example=input_example)


mse: 926605.9148228912
mae: 780.8403095415381
rmse 962.6037163978182
r2: -1.4212997929308262
🏃 View run rfwizimportfea_forprbaro at: http://localhost:5000/#/experiments/10/runs/ca88783841fe43a5ba010f8703d56ff3
🧪 View experiment at: http://localhost:5000/#/experiments/10


GradientBoostingRegression

In [5]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
# Train Gradient Boosting Regressor

model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(X_train, y_train)

with mlflow.start_run(run_name="gradientboosting_forprbaro"):
# Make predictions
    y_pred = model.predict(X_test)

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    mae=mean_absolute_error(y_test,y_pred)
    rmse=root_mean_squared_error(y_test,y_pred)
    r2=r2_score(y_test,y_pred)
    print("rmse:",rmse)
    print("mae:",mae)
    print("mse",mse)
    print("r2:",r2)

    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2",r2)
    mlflow.log_metric("mse",mse)
    mlflow.log_metric("rmse",rmse)
    

# Print the feature importances
#feature_importances = model.feature_importances_
#print("Feature Importances:", feature_importances)

rmse: 688.3478943030701
mae: 541.3330492639541
mse 473822.8235914706
r2: -0.2381391984392982
🏃 View run gradientboosting_forprbaro at: http://localhost:5000/#/experiments/10/runs/273ada0a6d83458fb1f5211bb15723eb
🧪 View experiment at: http://localhost:5000/#/experiments/10


In [23]:
# Train Gradient Boosting Regressor

model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(X_train, y_train)

with mlflow.start_run(run_name="gradientboostingnotgridsearchwithimportcol_forprbaro"):
# Make predictions
    y_pred = model.predict(X_test)

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    mae=mean_absolute_error(y_test,y_pred)
    rmse=root_mean_squared_error(y_test,y_pred)
    r2=r2_score(y_test,y_pred)
    print("rmse:",rmse)
    print("mae:",mae)
    print("mse",mse)
    print("r2:",r2)

    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2",r2)
    mlflow.log_metric("mse",mse)
    mlflow.log_metric("rmse",rmse)


rmse: 701.2823447660583
mae: 559.1719940099937
mse 491796.9270805807
r2: -0.285107054309955
🏃 View run gradientboostingnotgridsearchwithimportcol_forprbaro at: http://localhost:5000/#/experiments/10/runs/538395668f614b04a4b62a48fedf213a
🧪 View experiment at: http://localhost:5000/#/experiments/10


Gradient boosting with grid search and important features

In [21]:
len(X_train)

150420

In [22]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

# Initialize base model
base_model = GradientBoostingRegressor(random_state=42)

# Grid search
X_sample = X_train.sample(frac=0.2, random_state=42)
y_sample = y_train.loc[X_sample.index]

grid_search = GridSearchCV(base_model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_sample, y_sample)

# Best model
best_model = grid_search.best_estimator_

# Start MLflow run
with mlflow.start_run(run_name="gradientboosting_grids&importcols_forprbaro"):
    # Predict
    y_pred = best_model.predict(X_test)

    # Evaluate
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    print("Best Params:", grid_search.best_params_)
    print("RMSE:", rmse)
    print("MAE:", mae)
    print("MSE:", mse)
    print("R2:", r2)

    # Log metrics
    mlflow.log_params(grid_search.best_params_)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("rmse", rmse)


Best Params: {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 200}
RMSE: 993.777564881101
MAE: 754.4521774459381
MSE: 987593.8484610107
R2: -1.5806664327579454
🏃 View run gradientboosting_grids&importcols_forprbaro at: http://localhost:5000/#/experiments/10/runs/981b1b3281694cb9b912710b81647311
🧪 View experiment at: http://localhost:5000/#/experiments/10


## 2. For te_air_scav_rec 

In [17]:
#Define new X and y 
y=numeric_df["te_air_scav_rec"]
X =numeric_df.drop(columns=["te_air_scav_rec"])

In [18]:
total_samples=len(X)
test_ratio=0.2
test_size=int(total_samples*test_ratio)
n_splits=(total_samples-test_size)//test_size
tscv = TimeSeriesSplit(n_splits=n_splits,test_size=test_size,gap=0)

for train_idx, test_idx in tscv.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

2.1 Linear Regression

In [19]:
Pipeline1 = Pipeline([ ('scaler', StandardScaler()), 
                    ('Linear Regression', LinearRegression())])

Pipeline1.fit(X_train,y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('Linear Regression', LinearRegression())])

In [20]:
input_example=X_test.iloc[:1]
with mlflow.start_run(run_name="linear_for_te_air_scav_rec"):
    y_pred=Pipeline1.predict(X_test)
    mse=mean_squared_error(y_test,y_pred)
    rmse=root_mean_squared_error(y_test,y_pred)
    mae=mean_absolute_error(y_test,y_pred)
    r2=r2_score(y_test,y_pred)
    print("mse:",mse)
    print("rmse:",rmse)
    print("mae:",mae)
    print("r2:",r2)

    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2",r2)
    mlflow.log_metric("mse",mse)
    mlflow.log_metric("rmse",rmse)
    mlflow.sklearn.log_model(Pipeline1, artifact_path="linear_for_te_air_scav_rec", input_example=input_example)

mse: 0.24943357011188103
rmse: 0.49943324890507745
mae: 0.33686713197087265
r2: 0.962853788693097
🏃 View run linear_for_te_air_scav_rec at: http://localhost:5000/#/experiments/10/runs/5773422059bc49f7ac6e490c2f2c8c2a
🧪 View experiment at: http://localhost:5000/#/experiments/10


2.1.2 Linear with important features

In [ ]:
y=numeric_df["te_air_scav_rec"]
X =numeric_df.drop(columns=["te_air_scav_rec"])

2.2 Random Forest 

In [21]:
Pipeline2 = Pipeline([('Scaler', StandardScaler()), 
                    ('RandomForest', RandomForestRegressor())])

Pipeline2.fit(X_train,y_train)

with mlflow.start_run(run_name="rf_for_te_air_scav_rec"):
    y_pred=Pipeline2.predict(X_test)
    mse=mean_squared_error(y_test,y_pred)
    mae=mean_absolute_error(y_test,y_pred)
    rmse=root_mean_squared_error(y_test,y_pred)
    r2=r2_score(y_test,y_pred)
    print("mse:",mse)
    print("mae:",mae)
    print("rmse",rmse)
    print("r2:",r2)

    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2",r2)
    mlflow.log_metric("mse",mse)
    mlflow.log_metric("rmse",rmse)
    mlflow.sklearn.log_model(Pipeline2, artifact_path="rf_for_te_air_scav_rec", input_example=input_example)


mse: 1.530339757393545
mae: 0.9102108020425435
rmse 1.2370690188479967
r2: 0.7720983427611715
🏃 View run rf_for_te_air_scav_rec at: http://localhost:5000/#/experiments/10/runs/ffa36bbcdd8346b09c0cdc020552b9dc
🧪 View experiment at: http://localhost:5000/#/experiments/10


2.3 Gradient Boosting

In [22]:
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(X_train, y_train)

with mlflow.start_run(run_name="gradientboosting_for_te_air_scav_rec"):
# Make predictions
    y_pred = model.predict(X_test)

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    mae=mean_absolute_error(y_test,y_pred)
    rmse=root_mean_squared_error(y_test,y_pred)
    r2=r2_score(y_test,y_pred)
    print("rmse:",rmse)
    print("mae:",mae)
    print("mse",mse)
    print("r2:",r2)

    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2",r2)
    mlflow.log_metric("mse",mse)
    mlflow.log_metric("rmse",rmse)
    

rmse: 1.6498073119563907
mae: 1.1930430656523336
mse 2.7218641665847714
r2: 0.5946538333421012
🏃 View run gradientboosting_for_te_air_scav_rec at: http://localhost:5000/#/experiments/10/runs/89616eb8beca467da827326364182158
🧪 View experiment at: http://localhost:5000/#/experiments/10


## 3. For pr_exh_turb_out

In [4]:
df=pd.read_parquet("/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/df2_for_simplemodels.parquet")

In [7]:
df1=df[["pr_exh_turb_out__0","bo_aux_blower_running","te_air_ic_out__0_1","pd_air_ic__0","re_eng_load","pd_air_ic__0_1","pr_air_scav_ecs"]]

In [8]:
df1.columns

Index(['pr_exh_turb_out__0', 'bo_aux_blower_running', 'te_air_ic_out__0_1',
       'pd_air_ic__0', 're_eng_load', 'pd_air_ic__0_1', 'pr_air_scav_ecs'],
      dtype='object')

In [11]:
len(df1)

528749

In [10]:
print(len(df1["pr_exh_turb_out__0"].isna()))

528749


In [9]:
#drop Nans because the later processes don't accept nan values
df2=df1.dropna(axis=1,how="all")
df2=df2.dropna(axis=0,how="any")
numeric_df=df2.select_dtypes(include='number')
print(len(numeric_df),numeric_df.columns)

477962 Index(['bo_aux_blower_running', 're_eng_load', 'pr_air_scav_ecs'], dtype='object')


In [30]:
availability=pd.read_parquet("/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/han/database_data_availability_202404sfoc")

In [46]:
print(availability["te_exh_turb_in__0"][:10])

0    100.0
1    100.0
2    100.0
3    100.0
4    100.0
5    100.0
6    100.0
7    100.0
8    100.0
9    100.0
Name: te_exh_turb_in__0, dtype: float64


In [34]:
availability

,product_id,engine_type,time,te_exh_cyl_out__0,te_exh_cyl_out__1,te_exh_cyl_out__2,te_exh_cyl_out__3,te_exh_cyl_out__4,te_exh_cyl_out__5,te_exh_cyl_out__6,...,fr_tc__3,pr_baro,pr_exh_rec,pr_air_scav_ecs,te_air_scav_rec,te_exh_turb_in__0,te_exh_turb_in__1,te_exh_turb_in__2,te_exh_turb_in__3,re_eng_load
0,fd2923446b3caffe0455ad688a061386,S90ME-C9.2-TII,0.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,...,100.0,100.00000,100.0,0.037783,100.000000,100.0,100.0,100.0,100.0,68.384340
1,7822f5d4237c85e88337b1e4ebea960c,G70ME-C9.2-GI-TII,0.0,0.654739,0.654739,0.654739,0.654739,0.654739,100.000000,100.0,...,100.0,100.00000,100.0,0.247026,0.654739,100.0,100.0,100.0,100.0,30.412989
2,21f31851f82fe0a4060cdf7ded2fc146,G60ME-C9.5-LGIP,0.0,0.287349,0.287349,0.287349,0.289744,0.287349,0.289744,100.0,...,100.0,100.00000,100.0,0.141280,100.000000,100.0,100.0,100.0,100.0,8.986854
3,b4aa59577bab5a409f463181ebeeb108,G60ME-C9.5-LGIP-TII,0.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,...,100.0,100.00000,100.0,0.198774,100.000000,100.0,100.0,100.0,100.0,14.302136
4,e82ef34bc2b49f6ced5cecd1bf58b8e3,S50ME-B9.3-TII,0.0,0.141280,0.141280,0.141280,0.141280,0.141280,0.141280,100.0,...,100.0,100.00000,100.0,0.141280,0.141280,100.0,100.0,100.0,100.0,64.550657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,9274e1a9c8785c404d42f3babcc61b12,G70ME-C10.5-GI-HPSCR,0.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,...,100.0,0.14128,100.0,0.141280,100.000000,100.0,100.0,100.0,100.0,27.298197
484,173825f1a69d2d4e38eb51291684fcaf,G70ME-C10.5-GI-HPSCR,0.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,...,100.0,0.14128,100.0,0.141280,100.000000,100.0,100.0,100.0,100.0,16.730921
485,634ecf4f10fdc5d86393765fa2dcf61a,S50ME-C9.7-LPSCR,0.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,...,100.0,0.14128,100.0,0.141280,100.000000,100.0,100.0,100.0,100.0,35.609779
486,cdbf1d8060eec9498d8d998ed17e1764,G70ME-C9.5-GI-EGRBP,0.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,...,100.0,100.00000,100.0,0.143675,100.000000,100.0,100.0,100.0,100.0,27.331721


In [47]:
df3=pd.read_parquet("/home/ec2-user/SageMaker/sensor-imputation-thesis/src/sensor_imputation_thesis/nadire/df2_for_simplemodels.parquet")

In [58]:
print(df3["te_air_comp_in_a__0"].isna().sum())


749311


In [52]:
len(df3)

749311

In [53]:
df3.columns

Index(['time', 'fr_eng', 'te_exh_cyl_out__0', 'pd_air_ic__0',
       'pr_exh_turb_out__0', 'te_air_ic_out__0', 'te_seawater',
       'te_air_comp_in_a__0', 'te_air_comp_in_b__0', 'fr_tc__0', 'pr_baro',
       'pd_air_ic__0_1', 'pr_exh_rec', 'te_exh_turb_in__0',
       'te_exh_turb_out__0', 'bo_aux_blower_running', 're_eng_load',
       'pr_air_scav_ecs', 'pr_air_scav', 'te_air_scav_rec',
       'te_air_ic_out__0_1', 'pr_cyl_comp__0', 'pr_cyl_max__0', 'se_mip__0',
       'te_exh_cyl_out__0_1', 'fr_eng_setpoint', 'te_air_scav_rec_iso',
       'pr_cyl_max_mv_iso', 'pr_cyl_comp_mv_iso', 'fr_eng_ecs',
       'pr_air_scav_iso'],
      dtype='object')